In [1]:
import pandas as pd
from pathlib import Path
import re

In [2]:
DATA_DIR = Path("./../data")
RAW_FILE = DATA_DIR / "raw/Sales_of_real_estate_N_per_quarter_per_municipality_2010_2025.xlsx"
PROCESSED_FILE = DATA_DIR / "processed/Sales_of_real_estate_N_per_quarter_per_municipality_2010_2025_processed.xlsx"

In [3]:
xls = pd.ExcelFile(RAW_FILE)
xls.sheet_names[:10]

['Par commune']

In [4]:
# Load with two header rows
df_raw = pd.read_excel(RAW_FILE, sheet_name="Par commune", header=[0, 1, 2])

# Combine the multi-index columns into one string
df_raw.columns = [
    "_".join([str(c) for c in col if str(c) != "nan"]).strip().lower().replace(" ", "_")
    for col in df_raw.columns.values
]

display(df_raw.columns)
df_raw.head()

Index(['catégories_immobilières_basées_sur_la_nature_spécifiée_dans_l'acte_de_vente_unnamed:_0_level_1_refnis',
       'catégories_immobilières_basées_sur_la_nature_spécifiée_dans_l'acte_de_vente_unnamed:_1_level_1_localité',
       'catégories_immobilières_basées_sur_la_nature_spécifiée_dans_l'acte_de_vente_unnamed:_2_level_1_année',
       'catégories_immobilières_basées_sur_la_nature_spécifiée_dans_l'acte_de_vente_unnamed:_3_level_1_période',
       'catégories_immobilières_basées_sur_la_nature_spécifiée_dans_l'acte_de_vente_unnamed:_4_level_1_période',
       'ventes_de_biens_immobiliers_toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)_nombre_transactions',
       'ventes_de_biens_immobiliers_toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)_prix_médian(€)',
       'ventes_de_biens_immobiliers_toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)_prix_premier_quartile(€)',
       'ventes_de_biens_immobiliers_toutes_les_

,catégories_immobilières_basées_sur_la_nature_spécifiée_dans_l'acte_de_vente_unnamed:_0_level_1_refnis,catégories_immobilières_basées_sur_la_nature_spécifiée_dans_l'acte_de_vente_unnamed:_1_level_1_localité,catégories_immobilières_basées_sur_la_nature_spécifiée_dans_l'acte_de_vente_unnamed:_2_level_1_année,catégories_immobilières_basées_sur_la_nature_spécifiée_dans_l'acte_de_vente_unnamed:_3_level_1_période,catégories_immobilières_basées_sur_la_nature_spécifiée_dans_l'acte_de_vente_unnamed:_4_level_1_période,"ventes_de_biens_immobiliers_toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)_nombre_transactions","ventes_de_biens_immobiliers_toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)_prix_médian(€)","ventes_de_biens_immobiliers_toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)_prix_premier_quartile(€)","ventes_de_biens_immobiliers_toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)_prix_troisième_quartile(€)","ventes_de_biens_immobiliers_toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)_prix_troisième_quartile(€).1",...,ventes_de_biens_immobiliers_maisons_avec_4_ou_plus_de_façades_(type_ouvert)_prix_troisième_quartile(€),ventes_de_biens_immobiliers_maisons_avec_4_ou_plus_de_façades_(type_ouvert)_prix_troisième_quartile(€).1,ventes_de_biens_immobiliers_appartements_nombre_transactions,ventes_de_biens_immobiliers_appartements_prix_médian(€),ventes_de_biens_immobiliers_appartements_prix_premier_quartile(€),ventes_de_biens_immobiliers_appartements_prix_troisième_quartile(€),ventes_de_biens_immobiliers_appartements_prix_troisième_quartile(€).1,ventes_de_biens_immobiliers_appartements_prix_troisième_quartile(€).2,ventes_de_biens_immobiliers_appartements_prix_troisième_quartile(€).3,ventes_de_biens_immobiliers_appartements_0
0,11001,AARTSELAAR,2010,Q1,NaN,29.0,252000.0,225000.0,290000.0,NaN,...,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,1
1,11001,AARTSELAAR,2010,Q2,NaN,25.0,254000.0,212000.0,289000.0,NaN,...,NaN,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN,1
2,11001,AARTSELAAR,2010,Q3,NaN,21.0,255000.0,235000.0,285000.0,NaN,...,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,1
3,11001,AARTSELAAR,2010,Q4,NaN,28.0,245000.0,210000.0,312500.0,NaN,...,NaN,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,1
4,11001,AARTSELAAR,2011,Q1,NaN,23.0,310000.0,220000.0,370000.0,NaN,...,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,1


In [5]:
df = df_raw.copy()

# Clean key columns

In [6]:
rename_map = {
    "catégories_immobilières_basées_sur_la_nature_spécifiée_dans_l'acte_de_vente_unnamed:_0_level_1_refnis": "refnis",
    "catégories_immobilières_basées_sur_la_nature_spécifiée_dans_l'acte_de_vente_unnamed:_1_level_1_localité": "localite",
    "catégories_immobilières_basées_sur_la_nature_spécifiée_dans_l'acte_de_vente_unnamed:_2_level_1_année": "annee",
    "catégories_immobilières_basées_sur_la_nature_spécifiée_dans_l'acte_de_vente_unnamed:_3_level_1_période": "periode",
    "catégories_immobilières_basées_sur_la_nature_spécifiée_dans_l'acte_de_vente_unnamed:_4_level_1_période": "periode_dup",
}
df = df.rename(columns=rename_map)

# Drop the duplicate periode if present
if "periode_dup" in df.columns:
    df = df.drop(columns=["periode_dup"])

df.head()

,refnis,localite,annee,periode,"ventes_de_biens_immobiliers_toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)_nombre_transactions","ventes_de_biens_immobiliers_toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)_prix_médian(€)","ventes_de_biens_immobiliers_toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)_prix_premier_quartile(€)","ventes_de_biens_immobiliers_toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)_prix_troisième_quartile(€)","ventes_de_biens_immobiliers_toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)_prix_troisième_quartile(€).1",ventes_de_biens_immobiliers_maisons_avec_2_ou_3_façades_(type_fermé_+_type_demi-fermé)_nombre_transactions,...,ventes_de_biens_immobiliers_maisons_avec_4_ou_plus_de_façades_(type_ouvert)_prix_troisième_quartile(€),ventes_de_biens_immobiliers_maisons_avec_4_ou_plus_de_façades_(type_ouvert)_prix_troisième_quartile(€).1,ventes_de_biens_immobiliers_appartements_nombre_transactions,ventes_de_biens_immobiliers_appartements_prix_médian(€),ventes_de_biens_immobiliers_appartements_prix_premier_quartile(€),ventes_de_biens_immobiliers_appartements_prix_troisième_quartile(€),ventes_de_biens_immobiliers_appartements_prix_troisième_quartile(€).1,ventes_de_biens_immobiliers_appartements_prix_troisième_quartile(€).2,ventes_de_biens_immobiliers_appartements_prix_troisième_quartile(€).3,ventes_de_biens_immobiliers_appartements_0
0,11001,AARTSELAAR,2010,Q1,29.0,252000.0,225000.0,290000.0,NaN,25.0,...,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,1
1,11001,AARTSELAAR,2010,Q2,25.0,254000.0,212000.0,289000.0,NaN,20.0,...,NaN,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN,1
2,11001,AARTSELAAR,2010,Q3,21.0,255000.0,235000.0,285000.0,NaN,18.0,...,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,1
3,11001,AARTSELAAR,2010,Q4,28.0,245000.0,210000.0,312500.0,NaN,21.0,...,NaN,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,1
4,11001,AARTSELAAR,2011,Q1,23.0,310000.0,220000.0,370000.0,NaN,18.0,...,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,1


In [7]:
df.describe()

,refnis,annee,"ventes_de_biens_immobiliers_toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)_nombre_transactions","ventes_de_biens_immobiliers_toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)_prix_médian(€)","ventes_de_biens_immobiliers_toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)_prix_premier_quartile(€)","ventes_de_biens_immobiliers_toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)_prix_troisième_quartile(€)","ventes_de_biens_immobiliers_toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)_prix_troisième_quartile(€).1",ventes_de_biens_immobiliers_maisons_avec_2_ou_3_façades_(type_fermé_+_type_demi-fermé)_nombre_transactions,ventes_de_biens_immobiliers_maisons_avec_2_ou_3_façades_(type_fermé_+_type_demi-fermé)_prix_médian(€),ventes_de_biens_immobiliers_maisons_avec_2_ou_3_façades_(type_fermé_+_type_demi-fermé)_prix_premier_quartile(€),...,ventes_de_biens_immobiliers_maisons_avec_4_ou_plus_de_façades_(type_ouvert)_prix_troisième_quartile(€),ventes_de_biens_immobiliers_maisons_avec_4_ou_plus_de_façades_(type_ouvert)_prix_troisième_quartile(€).1,ventes_de_biens_immobiliers_appartements_nombre_transactions,ventes_de_biens_immobiliers_appartements_prix_médian(€),ventes_de_biens_immobiliers_appartements_prix_premier_quartile(€),ventes_de_biens_immobiliers_appartements_prix_troisième_quartile(€),ventes_de_biens_immobiliers_appartements_prix_troisième_quartile(€).1,ventes_de_biens_immobiliers_appartements_prix_troisième_quartile(€).2,ventes_de_biens_immobiliers_appartements_prix_troisième_quartile(€).3,ventes_de_biens_immobiliers_appartements_0
count,35030.000000,35030.000000,34946.000000,2.387700e+04,23877.000000,2.387700e+04,0.0,34447.000000,15163.000000,15163.000000,...,8.588000e+03,0.0,26058.000000,5697.000000,5697.000000,5697.000000,0.0,0.0,0.0,35030.000000
mean,47214.663717,2017.258065,38.943313,2.471206e+05,187969.107677,3.232595e+05,NaN,26.895027,223692.122601,174515.540262,...,4.355849e+05,NaN,21.790276,198068.629630,155117.849394,252891.906793,NaN,NaN,NaN,283.000000
std,24353.183803,4.479175,57.183636,9.870956e+04,78875.542697,1.383336e+05,NaN,52.469113,92222.957181,76187.911934,...,2.254835e+05,NaN,74.463274,55217.463768,43348.814303,78546.195403,NaN,NaN,NaN,163.103524
min,11001.000000,2010.000000,1.000000,3.500000e+04,14500.000000,6.700000e+04,NaN,1.000000,36250.000000,31750.000000,...,6.425000e+04,NaN,1.000000,35000.000000,18000.000000,53000.000000,NaN,NaN,NaN,1.000000
25%,24104.000000,2013.000000,14.000000,1.800000e+05,135000.000000,2.370000e+05,NaN,6.000000,162500.000000,122500.000000,...,3.120000e+05,NaN,2.000000,160000.000000,125000.000000,200000.000000,NaN,NaN,NaN,142.000000
50%,45062.000000,2017.000000,25.000000,2.300000e+05,175000.000000,2.990000e+05,NaN,14.000000,206000.000000,161500.000000,...,3.940000e+05,NaN,5.000000,190000.000000,150000.000000,242000.000000,NaN,NaN,NaN,283.000000
75%,63075.000000,2021.000000,45.000000,2.940000e+05,230000.000000,3.770000e+05,NaN,29.000000,265000.000000,210000.000000,...,4.975000e+05,NaN,14.000000,228750.000000,180000.000000,286250.000000,NaN,NaN,NaN,424.000000
max,93090.000000,2025.000000,1260.000000,1.200000e+06,750000.000000,2.275000e+06,NaN,1219.000000,950000.000000,750000.000000,...,3.931500e+06,NaN,2103.000000,600000.000000,400000.000000,930500.000000,NaN,NaN,NaN,565.000000


In [8]:
rename_map = {
    "catégories_immobilières_basées_sur_la_nature_spécifiée_dans_l'acte_de_vente_unnamed:_0_level_1_refnis": "refnis",
    "catégories_immobilières_basées_sur_la_nature_spécifiée_dans_l'acte_de_vente_unnamed:_1_level_1_localité": "localite",
    "catégories_immobilières_basées_sur_la_nature_spécifiée_dans_l'acte_de_vente_unnamed:_2_level_1_année": "annee",
    "catégories_immobilières_basées_sur_la_nature_spécifiée_dans_l'acte_de_vente_unnamed:_3_level_1_période": "periode",
    "catégories_immobilières_basées_sur_la_nature_spécifiée_dans_l'acte_de_vente_unnamed:_4_level_1_période": "periode_dup",
}
df = df.rename(columns=rename_map)

# Drop the duplicate periode if present
if "periode_dup" in df.columns:
    df = df.drop(columns=["periode_dup"])

df.drop(columns=[
    'ventes_de_biens_immobiliers_toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)_prix_troisième_quartile(€).1',
    'ventes_de_biens_immobiliers_maisons_avec_2_ou_3_façades_(type_fermé_+_type_demi-fermé)_prix_troisième_quartile(€).1',
    'ventes_de_biens_immobiliers_maisons_avec_4_ou_plus_de_façades_(type_ouvert)_prix_troisième_quartile(€).1',
    'ventes_de_biens_immobiliers_appartements_prix_troisième_quartile(€).1',
    'ventes_de_biens_immobiliers_appartements_prix_troisième_quartile(€).2',
    'ventes_de_biens_immobiliers_appartements_prix_troisième_quartile(€).3',
], inplace=True)

df.head()

,refnis,localite,annee,periode,"ventes_de_biens_immobiliers_toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)_nombre_transactions","ventes_de_biens_immobiliers_toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)_prix_médian(€)","ventes_de_biens_immobiliers_toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)_prix_premier_quartile(€)","ventes_de_biens_immobiliers_toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)_prix_troisième_quartile(€)",ventes_de_biens_immobiliers_maisons_avec_2_ou_3_façades_(type_fermé_+_type_demi-fermé)_nombre_transactions,ventes_de_biens_immobiliers_maisons_avec_2_ou_3_façades_(type_fermé_+_type_demi-fermé)_prix_médian(€),...,ventes_de_biens_immobiliers_maisons_avec_2_ou_3_façades_(type_fermé_+_type_demi-fermé)_prix_troisième_quartile(€),ventes_de_biens_immobiliers_maisons_avec_4_ou_plus_de_façades_(type_ouvert)_nombre_transactions,ventes_de_biens_immobiliers_maisons_avec_4_ou_plus_de_façades_(type_ouvert)_prix_médian(€),ventes_de_biens_immobiliers_maisons_avec_4_ou_plus_de_façades_(type_ouvert)_prix_premier_quartile(€),ventes_de_biens_immobiliers_maisons_avec_4_ou_plus_de_façades_(type_ouvert)_prix_troisième_quartile(€),ventes_de_biens_immobiliers_appartements_nombre_transactions,ventes_de_biens_immobiliers_appartements_prix_médian(€),ventes_de_biens_immobiliers_appartements_prix_premier_quartile(€),ventes_de_biens_immobiliers_appartements_prix_troisième_quartile(€),ventes_de_biens_immobiliers_appartements_0
0,11001,AARTSELAAR,2010,Q1,29.0,252000.0,225000.0,290000.0,25.0,247000.0,...,265000.0,4.0,NaN,NaN,NaN,9.0,NaN,NaN,NaN,1
1,11001,AARTSELAAR,2010,Q2,25.0,254000.0,212000.0,289000.0,20.0,251250.0,...,275000.0,5.0,NaN,NaN,NaN,11.0,NaN,NaN,NaN,1
2,11001,AARTSELAAR,2010,Q3,21.0,255000.0,235000.0,285000.0,18.0,250000.0,...,275000.0,3.0,NaN,NaN,NaN,9.0,NaN,NaN,NaN,1
3,11001,AARTSELAAR,2010,Q4,28.0,245000.0,210000.0,312500.0,21.0,245000.0,...,280000.0,7.0,NaN,NaN,NaN,13.0,NaN,NaN,NaN,1
4,11001,AARTSELAAR,2011,Q1,23.0,310000.0,220000.0,370000.0,18.0,280000.0,...,360000.0,5.0,NaN,NaN,NaN,10.0,NaN,NaN,NaN,1


# Melt to long format

In [9]:
df.columns

Index(['refnis', 'localite', 'annee', 'periode',
       'ventes_de_biens_immobiliers_toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)_nombre_transactions',
       'ventes_de_biens_immobiliers_toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)_prix_médian(€)',
       'ventes_de_biens_immobiliers_toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)_prix_premier_quartile(€)',
       'ventes_de_biens_immobiliers_toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)_prix_troisième_quartile(€)',
       'ventes_de_biens_immobiliers_maisons_avec_2_ou_3_façades_(type_fermé_+_type_demi-fermé)_nombre_transactions',
       'ventes_de_biens_immobiliers_maisons_avec_2_ou_3_façades_(type_fermé_+_type_demi-fermé)_prix_médian(€)',
       'ventes_de_biens_immobiliers_maisons_avec_2_ou_3_façades_(type_fermé_+_type_demi-fermé)_prix_premier_quartile(€)',
       'ventes_de_biens_immobiliers_maisons_avec_2_ou_3_façades_(type_fer

In [10]:
def melting(df, id_vars, value_vars):
    tmp_df = df[id_vars+value_vars].copy()
    tmp_df.head()
    
    df_long = df.melt(id_vars=id_vars, value_vars=value_vars,
                      var_name="variable", value_name="value")

    return df_long

In [11]:
id_vars = ["refnis", "localite", "annee", "periode", "ventes_de_biens_immobiliers_toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)_nombre_transactions"]
value_vars = [
    'ventes_de_biens_immobiliers_toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)_prix_médian(€)',
    'ventes_de_biens_immobiliers_toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)_prix_premier_quartile(€)',
    'ventes_de_biens_immobiliers_toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)_prix_troisième_quartile(€)',
]

df_long_1 = melting(df, id_vars, value_vars)

df_long_1.rename(columns={"ventes_de_biens_immobiliers_toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)_nombre_transactions": "nombre_transactions"}, inplace=True)

df_long_1.head()

,refnis,localite,annee,periode,nombre_transactions,variable,value
0,11001,AARTSELAAR,2010,Q1,29.0,ventes_de_biens_immobiliers_toutes_les_maisons...,252000.0
1,11001,AARTSELAAR,2010,Q2,25.0,ventes_de_biens_immobiliers_toutes_les_maisons...,254000.0
2,11001,AARTSELAAR,2010,Q3,21.0,ventes_de_biens_immobiliers_toutes_les_maisons...,255000.0
3,11001,AARTSELAAR,2010,Q4,28.0,ventes_de_biens_immobiliers_toutes_les_maisons...,245000.0
4,11001,AARTSELAAR,2011,Q1,23.0,ventes_de_biens_immobiliers_toutes_les_maisons...,310000.0


In [12]:
id_vars = ["refnis", "localite", "annee", "periode", "ventes_de_biens_immobiliers_maisons_avec_2_ou_3_façades_(type_fermé_+_type_demi-fermé)_nombre_transactions"]
value_vars = [
    'ventes_de_biens_immobiliers_maisons_avec_2_ou_3_façades_(type_fermé_+_type_demi-fermé)_prix_médian(€)',
    'ventes_de_biens_immobiliers_maisons_avec_2_ou_3_façades_(type_fermé_+_type_demi-fermé)_prix_premier_quartile(€)',
    'ventes_de_biens_immobiliers_maisons_avec_2_ou_3_façades_(type_fermé_+_type_demi-fermé)_prix_troisième_quartile(€)',
]

df_long_2 = melting(df, id_vars, value_vars)

df_long_2.rename(columns={"ventes_de_biens_immobiliers_maisons_avec_2_ou_3_façades_(type_fermé_+_type_demi-fermé)_nombre_transactions": "nombre_transactions"}, inplace=True)

df_long_2.head()

,refnis,localite,annee,periode,nombre_transactions,variable,value
0,11001,AARTSELAAR,2010,Q1,25.0,ventes_de_biens_immobiliers_maisons_avec_2_ou_...,247000.0
1,11001,AARTSELAAR,2010,Q2,20.0,ventes_de_biens_immobiliers_maisons_avec_2_ou_...,251250.0
2,11001,AARTSELAAR,2010,Q3,18.0,ventes_de_biens_immobiliers_maisons_avec_2_ou_...,250000.0
3,11001,AARTSELAAR,2010,Q4,21.0,ventes_de_biens_immobiliers_maisons_avec_2_ou_...,245000.0
4,11001,AARTSELAAR,2011,Q1,18.0,ventes_de_biens_immobiliers_maisons_avec_2_ou_...,280000.0


In [13]:
id_vars = ["refnis", "localite", "annee", "periode", "ventes_de_biens_immobiliers_maisons_avec_4_ou_plus_de_façades_(type_ouvert)_nombre_transactions"]
value_vars = [
    'ventes_de_biens_immobiliers_maisons_avec_4_ou_plus_de_façades_(type_ouvert)_prix_médian(€)',
    'ventes_de_biens_immobiliers_maisons_avec_4_ou_plus_de_façades_(type_ouvert)_prix_premier_quartile(€)',
    'ventes_de_biens_immobiliers_maisons_avec_4_ou_plus_de_façades_(type_ouvert)_prix_troisième_quartile(€)',
]

df_long_3 = melting(df, id_vars, value_vars)

df_long_3.rename(columns={"ventes_de_biens_immobiliers_maisons_avec_4_ou_plus_de_façades_(type_ouvert)_nombre_transactions": "nombre_transactions"}, inplace=True)

display(df_long_3.head())
df_long_3.describe()

,refnis,localite,annee,periode,nombre_transactions,variable,value
0,11001,AARTSELAAR,2010,Q1,4.0,ventes_de_biens_immobiliers_maisons_avec_4_ou_...,NaN
1,11001,AARTSELAAR,2010,Q2,5.0,ventes_de_biens_immobiliers_maisons_avec_4_ou_...,NaN
2,11001,AARTSELAAR,2010,Q3,3.0,ventes_de_biens_immobiliers_maisons_avec_4_ou_...,NaN
3,11001,AARTSELAAR,2010,Q4,7.0,ventes_de_biens_immobiliers_maisons_avec_4_ou_...,NaN
4,11001,AARTSELAAR,2011,Q1,5.0,ventes_de_biens_immobiliers_maisons_avec_4_ou_...,NaN


,refnis,annee,nombre_transactions,value
count,105090.000000,105090.000000,101325.000000,2.576400e+04
mean,47214.663717,2017.258065,12.863360,3.429522e+05
std,24352.952063,4.479132,11.152755,1.791779e+05
min,11001.000000,2010.000000,1.000000,1.700000e+04
25%,24104.000000,2013.000000,5.000000,2.400000e+05
50%,45062.000000,2017.000000,10.000000,3.110000e+05
75%,63075.000000,2021.000000,17.000000,4.025000e+05
max,93090.000000,2025.000000,153.000000,3.931500e+06


In [14]:
id_vars = ["refnis", "localite", "annee", "periode", "ventes_de_biens_immobiliers_appartements_nombre_transactions"]
value_vars = [
    'ventes_de_biens_immobiliers_appartements_prix_médian(€)',
    'ventes_de_biens_immobiliers_appartements_prix_premier_quartile(€)',
    'ventes_de_biens_immobiliers_appartements_prix_troisième_quartile(€)',
]

df_long_4 = melting(df, id_vars, value_vars)

df_long_4.rename(columns={"ventes_de_biens_immobiliers_appartements_nombre_transactions": "nombre_transactions"}, inplace=True)

display(df_long_4.head())
df_long_4.describe()

,refnis,localite,annee,periode,nombre_transactions,variable,value
0,11001,AARTSELAAR,2010,Q1,9.0,ventes_de_biens_immobiliers_appartements_prix_...,NaN
1,11001,AARTSELAAR,2010,Q2,11.0,ventes_de_biens_immobiliers_appartements_prix_...,NaN
2,11001,AARTSELAAR,2010,Q3,9.0,ventes_de_biens_immobiliers_appartements_prix_...,NaN
3,11001,AARTSELAAR,2010,Q4,13.0,ventes_de_biens_immobiliers_appartements_prix_...,NaN
4,11001,AARTSELAAR,2011,Q1,10.0,ventes_de_biens_immobiliers_appartements_prix_...,NaN


,refnis,annee,nombre_transactions,value
count,105090.000000,105090.000000,78174.000000,17091.000000
mean,47214.663717,2017.258065,21.790276,202026.128606
std,24352.952063,4.479132,74.462321,72800.986729
min,11001.000000,2010.000000,1.000000,18000.000000
25%,24104.000000,2013.000000,2.000000,153000.000000
50%,45062.000000,2017.000000,5.000000,190000.000000
75%,63075.000000,2021.000000,14.000000,240000.000000
max,93090.000000,2025.000000,2103.000000,930500.000000


In [15]:
df_long = pd.concat([df_long_1, df_long_2, df_long_3, df_long_4], ignore_index=True)
df_long

,refnis,localite,annee,periode,nombre_transactions,variable,value
0,11001,AARTSELAAR,2010,Q1,29.0,ventes_de_biens_immobiliers_toutes_les_maisons...,252000.0
1,11001,AARTSELAAR,2010,Q2,25.0,ventes_de_biens_immobiliers_toutes_les_maisons...,254000.0
2,11001,AARTSELAAR,2010,Q3,21.0,ventes_de_biens_immobiliers_toutes_les_maisons...,255000.0
3,11001,AARTSELAAR,2010,Q4,28.0,ventes_de_biens_immobiliers_toutes_les_maisons...,245000.0
4,11001,AARTSELAAR,2011,Q1,23.0,ventes_de_biens_immobiliers_toutes_les_maisons...,310000.0
...,...,...,...,...,...,...,...
420355,93090,VIROINVAL,2024,Q2,6.0,ventes_de_biens_immobiliers_appartements_prix_...,NaN
420356,93090,VIROINVAL,2024,Q3,NaN,ventes_de_biens_immobiliers_appartements_prix_...,NaN
420357,93090,VIROINVAL,2024,Q4,NaN,ventes_de_biens_immobiliers_appartements_prix_...,NaN
420358,93090,VIROINVAL,2025,Q1,NaN,ventes_de_biens_immobiliers_appartements_prix_...,NaN


# Extract property_type and metric

In [16]:
def split_var(v):
    # Remove prefix
    v = v.replace("ventes_de_biens_immobiliers_", "")
    # Property type is text before last "_"
    # Metric is last chunk after "_"
    # Example: "appartements_prix_médian(€)"
    # We'll do regex to separate
    m = re.match(r"(.+?)_((nombre_transactions)|(prix.*))$", v)
    if m:
        return m.group(1), m.group(2)
    else:
        return v, None

df_long[["property_type", "metric"]] = df_long["variable"].apply(lambda x: pd.Series(split_var(x)))

df_long.head()

,refnis,localite,annee,periode,nombre_transactions,variable,value,property_type,metric
0,11001,AARTSELAAR,2010,Q1,29.0,ventes_de_biens_immobiliers_toutes_les_maisons...,252000.0,"toutes_les_maisons_avec_2,_3,_4_ou_plus_de_faç...",prix_médian(€)
1,11001,AARTSELAAR,2010,Q2,25.0,ventes_de_biens_immobiliers_toutes_les_maisons...,254000.0,"toutes_les_maisons_avec_2,_3,_4_ou_plus_de_faç...",prix_médian(€)
2,11001,AARTSELAAR,2010,Q3,21.0,ventes_de_biens_immobiliers_toutes_les_maisons...,255000.0,"toutes_les_maisons_avec_2,_3,_4_ou_plus_de_faç...",prix_médian(€)
3,11001,AARTSELAAR,2010,Q4,28.0,ventes_de_biens_immobiliers_toutes_les_maisons...,245000.0,"toutes_les_maisons_avec_2,_3,_4_ou_plus_de_faç...",prix_médian(€)
4,11001,AARTSELAAR,2011,Q1,23.0,ventes_de_biens_immobiliers_toutes_les_maisons...,310000.0,"toutes_les_maisons_avec_2,_3,_4_ou_plus_de_faç...",prix_médian(€)


# Build datetime

In [17]:
df_long["quarter_str"] = df_long["annee"].astype(str) + df_long["periode"].astype(str)
df_long["date"] = pd.PeriodIndex(df_long["quarter_str"], freq="Q").to_timestamp("Q")

df_long.head()


,refnis,localite,annee,periode,nombre_transactions,variable,value,property_type,metric,quarter_str,date
0,11001,AARTSELAAR,2010,Q1,29.0,ventes_de_biens_immobiliers_toutes_les_maisons...,252000.0,"toutes_les_maisons_avec_2,_3,_4_ou_plus_de_faç...",prix_médian(€),2010Q1,2010-03-31
1,11001,AARTSELAAR,2010,Q2,25.0,ventes_de_biens_immobiliers_toutes_les_maisons...,254000.0,"toutes_les_maisons_avec_2,_3,_4_ou_plus_de_faç...",prix_médian(€),2010Q2,2010-06-30
2,11001,AARTSELAAR,2010,Q3,21.0,ventes_de_biens_immobiliers_toutes_les_maisons...,255000.0,"toutes_les_maisons_avec_2,_3,_4_ou_plus_de_faç...",prix_médian(€),2010Q3,2010-09-30
3,11001,AARTSELAAR,2010,Q4,28.0,ventes_de_biens_immobiliers_toutes_les_maisons...,245000.0,"toutes_les_maisons_avec_2,_3,_4_ou_plus_de_faç...",prix_médian(€),2010Q4,2010-12-31
4,11001,AARTSELAAR,2011,Q1,23.0,ventes_de_biens_immobiliers_toutes_les_maisons...,310000.0,"toutes_les_maisons_avec_2,_3,_4_ou_plus_de_faç...",prix_médian(€),2011Q1,2011-03-31


In [18]:
df_long.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420360 entries, 0 to 420359
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   refnis               420360 non-null  int64         
 1   localite             420360 non-null  object        
 2   annee                420360 non-null  int64         
 3   periode              420360 non-null  object        
 4   nombre_transactions  387678 non-null  float64       
 5   variable             420360 non-null  object        
 6   value                159975 non-null  float64       
 7   property_type        420360 non-null  object        
 8   metric               420360 non-null  object        
 9   quarter_str          420360 non-null  object        
 10  date                 420360 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(2), object(6)
memory usage: 35.3+ MB


# clean variable column

In [19]:
df_long.variable.value_counts()

variable
ventes_de_biens_immobiliers_toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)_prix_médian(€)                35030
ventes_de_biens_immobiliers_toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)_prix_premier_quartile(€)      35030
ventes_de_biens_immobiliers_toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)_prix_troisième_quartile(€)    35030
ventes_de_biens_immobiliers_maisons_avec_2_ou_3_façades_(type_fermé_+_type_demi-fermé)_prix_médian(€)                             35030
ventes_de_biens_immobiliers_maisons_avec_2_ou_3_façades_(type_fermé_+_type_demi-fermé)_prix_premier_quartile(€)                   35030
ventes_de_biens_immobiliers_maisons_avec_2_ou_3_façades_(type_fermé_+_type_demi-fermé)_prix_troisième_quartile(€)                 35030
ventes_de_biens_immobiliers_maisons_avec_4_ou_plus_de_façades_(type_ouvert)_prix_médian(€)                                        35030
ventes_de_biens_immobiliers_maisons_ave

In [20]:
def clean_var(v):
    # Remove prefix
    v = v.replace("ventes_de_biens_immobiliers_", "")
    return v

df_long["variable"] = df_long["variable"].apply(lambda x: pd.Series(clean_var(x)))

df_long.head()

,refnis,localite,annee,periode,nombre_transactions,variable,value,property_type,metric,quarter_str,date
0,11001,AARTSELAAR,2010,Q1,29.0,"toutes_les_maisons_avec_2,_3,_4_ou_plus_de_faç...",252000.0,"toutes_les_maisons_avec_2,_3,_4_ou_plus_de_faç...",prix_médian(€),2010Q1,2010-03-31
1,11001,AARTSELAAR,2010,Q2,25.0,"toutes_les_maisons_avec_2,_3,_4_ou_plus_de_faç...",254000.0,"toutes_les_maisons_avec_2,_3,_4_ou_plus_de_faç...",prix_médian(€),2010Q2,2010-06-30
2,11001,AARTSELAAR,2010,Q3,21.0,"toutes_les_maisons_avec_2,_3,_4_ou_plus_de_faç...",255000.0,"toutes_les_maisons_avec_2,_3,_4_ou_plus_de_faç...",prix_médian(€),2010Q3,2010-09-30
3,11001,AARTSELAAR,2010,Q4,28.0,"toutes_les_maisons_avec_2,_3,_4_ou_plus_de_faç...",245000.0,"toutes_les_maisons_avec_2,_3,_4_ou_plus_de_faç...",prix_médian(€),2010Q4,2010-12-31
4,11001,AARTSELAAR,2011,Q1,23.0,"toutes_les_maisons_avec_2,_3,_4_ou_plus_de_faç...",310000.0,"toutes_les_maisons_avec_2,_3,_4_ou_plus_de_faç...",prix_médian(€),2011Q1,2011-03-31


In [21]:
df_long.variable.value_counts()

variable
toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)_prix_médian(€)                35030
toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)_prix_premier_quartile(€)      35030
toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)_prix_troisième_quartile(€)    35030
maisons_avec_2_ou_3_façades_(type_fermé_+_type_demi-fermé)_prix_médian(€)                             35030
maisons_avec_2_ou_3_façades_(type_fermé_+_type_demi-fermé)_prix_premier_quartile(€)                   35030
maisons_avec_2_ou_3_façades_(type_fermé_+_type_demi-fermé)_prix_troisième_quartile(€)                 35030
maisons_avec_4_ou_plus_de_façades_(type_ouvert)_prix_médian(€)                                        35030
maisons_avec_4_ou_plus_de_façades_(type_ouvert)_prix_premier_quartile(€)                              35030
maisons_avec_4_ou_plus_de_façades_(type_ouvert)_prix_troisième_quartile(€)                            35030
appartements_prix_m

# result of cleaning

In [22]:
df_long.describe()

,refnis,annee,nombre_transactions,value,date
count,420360.000000,420360.000000,387678.000000,1.599750e+05,420360
mean,47214.663717,2017.258065,25.456464,2.544182e+05,2017-11-14 10:03:52.258064384
min,11001.000000,2010.000000,1.000000,1.450000e+04,2010-03-31 00:00:00
25%,24104.000000,2013.000000,6.000000,1.700000e+05,2013-12-31 00:00:00
50%,45062.000000,2017.000000,13.000000,2.300000e+05,2017-11-15 00:00:00
75%,63075.000000,2021.000000,28.000000,3.087500e+05,2021-09-30 00:00:00
max,93090.000000,2025.000000,2103.000000,3.931500e+06,2025-06-30 00:00:00
std,24352.865162,4.479116,53.502792,1.319479e+05,NaN


In [23]:
df_long[["localite", "periode", "variable", "property_type", "quarter_str", "metric"]].describe(include="all")

,localite,periode,variable,property_type,quarter_str,metric
count,420360,420360,420360,420360,420360,420360
unique,564,4,12,4,62,3
top,SAINT-NICOLAS,Q1,"toutes_les_maisons_avec_2,_3,_4_ou_plus_de_faç...","toutes_les_maisons_avec_2,_3,_4_ou_plus_de_faç...",2010Q1,prix_médian(€)
freq,1488,108480,35030,105090,6780,140120


# save the processed file

In [24]:
# could have saved in "parquet" for better performance but just for sake of consistency, I used Excel format again to save the cleaned dataframe
df_long.to_excel(PROCESSED_FILE, sheet_name="Par commune", index=False)